In [2]:
import sys
sys.path.append("/home/jovyan/work/sem-covid/")
sys.path = list(set(sys.path))

import os
os.getcwd()
os.chdir('/home/jovyan/work/sem-covid/')
from sem_covid import config
from sem_covid.services.store_registry import store_registry
import pandas as pd

In [1]:
MINIO_BUCKET = 'rdf-transformer'

In [3]:
df = store_registry.es_index_store().get_dataframe(index_name='ds_unified_dataset')

100% (6360 of 6360) |####################| Elapsed Time: 0:00:15 Time:  0:00:15


In [35]:
import pycountry

def get_country_iso3_code(country_name:str)->str:
    if country_name == 'European Union':
        return 'EUR'
    else:
        return pycountry.countries.get(name=country_name).alpha_3

In [55]:

def default_uri_generator(field_name:str, index: int, field_values: list):
    return f"{field_name}/category_{index}"

def country_uri_generator(field_name:str, index: int , field_values: list):
    return f"http://publications.europa.eu/resource/authority/country/{get_country_iso3_code(field_values[index])}"

In [53]:
def export_categorical_field_from_dataset(dataset: pd.DataFrame, field_name: str, uri_generator: callable) -> dict:
    field_values = list(set(dataset[field_name].explode().values))
    result = {}
    result[field_name] = [ {'uri': uri_generator(field_name,index,field_values), "name": str(field_values[index])}
    for index in range(0,len(field_values))]
    return result

In [59]:
import json

def export_categorical_fields_from_dataset(dataset: pd.DataFrame, field_names: list):
    minio = store_registry.minio_object_store(minio_bucket=MINIO_BUCKET)
    for field_name in field_names:
        tmp_dict = {}
        if field_name == 'country':
            tmp_dict = export_categorical_field_from_dataset(dataset=dataset, field_name=field_name, uri_generator=country_uri_generator)
        else:
            tmp_dict = export_categorical_field_from_dataset(dataset=dataset, field_name=field_name, uri_generator=default_uri_generator)
        tmp_json = json.dumps(tmp_dict)
        file_name = f"{field_name}.json"
        minio.put_object(object_name=file_name, content=tmp_json)

In [74]:
DS_UNIFIED_DATASET_CATEGORICAL_FIELDS = ['country', 'pwdb_category', 'pwdb_target_group_l1',
       'pwdb_funding', 'pwdb_type_of_measure', 'pwdb_actors','eu_cellar_subject_matter_labels','eu_cellar_resource_type_labels',
       'eu_cellar_author_labels', 'pwdb_target_group_l2', 'ireland_keyword', 'ireland_page_type', 'eu_timeline_topic'
       ]

In [75]:
export_categorical_fields_from_dataset(dataset=df, field_names= DS_UNIFIED_DATASET_CATEGORICAL_FIELDS)